# Chap09 分类数据

In [1]:
import pandas as pd
import numpy as np

## cat对象
### cat对象的属性
1. pandas中提供category类型，在一个分类类型的Series中定义了**cat**对象
2. `s.cat.categories`类别本身，以Index类型存储
3. `s.cat.ordered`类别是否有序，返回布尔值
4. 每一个序列的类别会被赋予唯一的整数编号，编号取决于`s.cat.categories`中的顺序，该属性可以通过`s.cat.codes`访问
### 类别的增加、删除和修改
1. 类别的增加`s.cat.add_categories('增加的类别')`
2. 类别的删除`s.cat.remove_categories('删除的类别')`，删除后原来序列中的该类会被设置为缺失
3. `set_categories`直接设置序列的新类别，原来的类别中如果存在元素不属于新类别，那么会被设置为缺失
4. `remove_unused_categories()`删除未出现在序列中的类别
5. `rename_categories({'原序列某类名':'修改后类名'})`修改后对原序列的对应值也进行相应修改

In [2]:
df = pd.read_csv('./data/learn_pandas.csv', usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight'])
s = df.Grade.astype('category')
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Junior', 'Senior', 'Sophomore']

In [5]:
s.cat.codes.head()

0    0
1    0
2    2
3    3
4    3
dtype: int8

## 有序分类
### 序的建立
1. 无序类别转化为有序类别`reorder_categories(排序后的类别列表,ordered=True)`传入的排序后的类别列表必须时由当前序列的所有**无序类别**构成的列表，不能多也不能少，且必须指定参数`ordered=True`
   - 不指定参数`ordered=True`的前提是先用`s.cat.as_ordered()`转化为有序类别，再利用`reorder_categories`进行具体的相对大小调整
2. 有序类别转化为无序类别`as_unordered()`
### 排序和比较
1. 把列的类型修改为category后，再赋予相应的大小关系，就能正常地使用`sort_index`和`sort_value`

## 区间类别
### 利用cut和qcut进行区间构造
1. 区间是一种特殊的category
2. `cut(序列s,bins,right,labels,retbins)`
   - 区间端点地调整为`0.001*(max-min)`
   - `bins`把整个传入数组地按照最大和最小值等间距地分为n段，或者指定区间分割点地列表，其中`np.infty`表示无穷大，`-np.infty`表示无穷小
   - `right`区间默认左开右闭，需要指定左闭右开时，指定`right=False`
   - `labels`区间的名字（即不显示(n1,n2])
   - `retbins`是否返回分割点，默认不返回
3. `qcut`
   - 用法上和`cut`几乎没差别，只是把bins参数变成q参数，表示按照n等分位数把数据分箱，还可以传入**浮点列表**指代相应地分位数分割点
### 一般区间的构造
1. 一个具体的区间具备三个要素：左端点、右端点和端点的开闭状态，开闭状态可以指定`right/left/both/neither`中的一类
2. 包含以下属性
   - `mid`中点
   - `length`长度
   - `right`右端点
   - `left`左端点
   - `closed`开闭状态
3. 使用`in`可以判断元素是否属于区间
4. 使用`overlaps`可以判断两个区间是否有交集
5. `pd.IntervalIndex`对象有四类方法
   - `from_breaks`功能类似`cut`或`qcut`，后两个是通过计算得到的分割点，前者是直接传入自定义的分割点
   - `from_arrays`是分别传入左端点和右端点的列表，适用于有交集并且知道起点和终点的情况
   - `from_tuples`传入的是起点和终点元组构成的列表
   - `interval_range`中的start确认等差区间序列的起点，end确认终点，periods确认区间个数，freq确认区间长度，这四个量只需要确定三个，剩下一个就可以确定
6. `pd.IntervalIndex([...], closed=...)`把Interval类型的列表组成传入其中转为区间索引，那么所有的区间会被强制转为指定的`closed`类型，因为`pd.IntervalIndex`只允许存放同一种开闭区间的Interval对象

In [9]:
s = pd.Series([1,2])
res = pd.cut(s, bins=2, labels=['small','big'], retbins=True)
res[0]

0    small
1      big
dtype: category
Categories (2, object): ['small' < 'big']

In [10]:
res[1]

array([0.999, 1.5  , 2.   ])

In [11]:
s = df.Weight
pd.qcut(s, q=[0,0.2,0.8,1]).head()

0      (44.0, 69.4]
1      (69.4, 89.0]
2      (69.4, 89.0]
3    (33.999, 44.0]
4      (69.4, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64, right]): [(33.999, 44.0] < (44.0, 69.4] < (69.4, 89.0]]

In [4]:
my_interval = pd.Interval(0,1,'right')
my_interval

Interval(0, 1, closed='right')

In [5]:
1 in my_interval

True

In [6]:
my_interval_2 = pd.Interval(0.5, 1.5, 'left')
my_interval.overlaps(my_interval_2)

True

In [7]:
pd.IntervalIndex.from_breaks([1,3,6,10], closed = 'both')

IntervalIndex([[1, 3], [3, 6], [6, 10]], dtype='interval[int64, both]')

In [8]:
pd.IntervalIndex.from_arrays(left = [1,3,6,10], right = [5,4,9,11], closed = 'neither')

IntervalIndex([(1, 5), (3, 4), (6, 9), (10, 11)], dtype='interval[int64, neither]')

In [9]:
pd.IntervalIndex.from_tuples([(1,5),(3,4),(6,9),(10,11)], closed='neither')

IntervalIndex([(1, 5), (3, 4), (6, 9), (10, 11)], dtype='interval[int64, neither]')

In [11]:
pd.interval_range(start=1, end=5, periods=8)

IntervalIndex([(1.0, 1.5], (1.5, 2.0], (2.0, 2.5], (2.5, 3.0], (3.0, 3.5], (3.5, 4.0], (4.0, 4.5], (4.5, 5.0]], dtype='interval[float64, right]')

In [12]:
my_interval

Interval(0, 1, closed='right')

In [13]:
my_interval_2

Interval(0.5, 1.5, closed='left')

In [14]:
pd.IntervalIndex([my_interval, my_interval_2], closed='left')

IntervalIndex([[0.0, 1.0), [0.5, 1.5)], dtype='interval[float64, left]')

## 区间的属性与方法
1. `IntervalIndex`上定义了一些有用的属性和方法，如果想要具体利用cut或qcut的结果进行分析，需要先将其转为该种索引类型
2. 若干常用属性：
   - `left`表示左端点
   - `right`表示右端点
   - `mid`表示两端点均值
   - `length`表示区间长度
3. 常用的两个方法：
   - `contains`逐个判断每个区间是否包含某元素
   - `overlaps`逐个判断每个区间和一个pd.Interval对象有交集

In [16]:
s = df.Weight
id_interval = pd.IntervalIndex(pd.cut(s,3))
id_interval[:3]

IntervalIndex([(33.945, 52.333], (52.333, 70.667], (70.667, 89.0]], dtype='interval[float64, right]', name='Weight')

In [17]:
id_demo = id_interval[:5]
id_demo

IntervalIndex([(33.945, 52.333], (52.333, 70.667], (70.667, 89.0], (33.945, 52.333], (70.667, 89.0]], dtype='interval[float64, right]', name='Weight')

In [18]:
id_demo.left

Float64Index([33.945, 52.333, 70.667, 33.945, 70.667], dtype='float64')

In [19]:
id_demo.right

Float64Index([52.333, 70.667, 89.0, 52.333, 89.0], dtype='float64')

In [20]:
id_demo.mid

Float64Index([43.138999999999996, 61.5, 79.8335, 43.138999999999996, 79.8335], dtype='float64')

In [21]:
id_demo.length

Float64Index([18.387999999999998, 18.334000000000003, 18.333,
              18.387999999999998, 18.333],
             dtype='float64')

In [22]:
id_demo.contains(4)

array([False, False, False, False, False])

In [23]:
id_demo.overlaps(pd.Interval(40,60))

array([ True,  True, False,  True, False])